In [2]:
import pyspark
import os
from pyspark import SQLContext
from pyspark import SparkContext 
from pyspark.sql import SQLContext 
sc = pyspark.SparkContext('local')
sqlContext = SQLContext(sc)

In [3]:
ratings = sc.textFile('/Users/drakaris/Downloads/final_data_ratings.csv').map(lambda line: line.split(","))
df = ratings.toDF(['userID','itemID','rating'])
df.show()

+-------+------+------+
| userID|itemID|rating|
+-------+------+------+
|1055480|  2485|   3.0|
|3305480|  2485|   5.0|
|1299050|     0|   5.0|
|1889139|     0|   5.0|
|2148308|     0|   4.0|
|2290394|     0|   5.0|
|2808467|     0|   5.0|
|2926918|     0|   5.0|
|3068052|     0|   4.0|
|4175374|     0|   4.0|
| 186478|  2486|   4.0|
| 306580|     1|   3.0|
| 330139|     1|   5.0|
|3138622|     1|   5.0|
|3397460|     2|   5.0|
|1959850|     3|   1.0|
|2030495|     3|   1.0|
|3679723|     3|   5.0|
|3844869|     3|   5.0|
|4293552|     3|   5.0|
+-------+------+------+
only showing top 20 rows



In [4]:
from pyspark.mllib.recommendation\
    import ALS,MatrixFactorizationModel, Rating
    
ratings = df.rdd.map(lambda x: Rating(int(x[0]),\
    int(x[1]), float(x[2])))    

train, test = ratings.randomSplit([0.8,0.2])


#print(train.count()) #70,005
#print(test.count()) #29,995

In [ ]:
#Need to cache the data to speed up training
train.cache()
#test.cache()

#Setting up the parameters for ALS
rank = 25 # Latent Factors to be made
numIterations = 25# Times to repeat process
regParam = 0.25
#Create the model on the training data
model = ALS.train(train, rank, numIterations, regParam)

In [103]:
model.recommendUsers(242,10)

[Rating(user=3331407, product=242, rating=4.8915102069922325),
 Rating(user=1830306, product=242, rating=4.180018835981711),
 Rating(user=3574972, product=242, rating=4.120627635856609),
 Rating(user=964706, product=242, rating=4.108605429308028),
 Rating(user=850892, product=242, rating=4.072058720045055),
 Rating(user=517372, product=242, rating=4.056570286076549),
 Rating(user=2788904, product=242, rating=4.050941986311813),
 Rating(user=2121867, product=242, rating=4.042190674932187),
 Rating(user=2652087, product=242, rating=4.028061222754539),
 Rating(user=3385645, product=242, rating=4.0177219143218)]

In [104]:
# For User Y Find N Products to Promote
model.recommendProducts(196,10)



[Rating(user=196, product=480725, rating=3.27080243959121),
 Rating(user=196, product=311825, rating=3.2210044450161686),
 Rating(user=196, product=145102, rating=3.1303897918023305),
 Rating(user=196, product=182403, rating=3.0771470381538286),
 Rating(user=196, product=279194, rating=3.067541446261095),
 Rating(user=196, product=82752, rating=3.054629310602661),
 Rating(user=196, product=190229, rating=3.0413355339168002),
 Rating(user=196, product=66167, rating=3.030873198958532),
 Rating(user=196, product=268497, rating=3.0257430276342934),
 Rating(user=196, product=33068, rating=3.015322049363553)]

In [105]:
#Predict Single Product for Single User
model.predict(196, 242)



1.3389621847276678

In [106]:
# Predict Multi Users and Multi Products
# Pre-Processing
pred_input = train.map(lambda x:(x[0],x[1]))   



In [108]:
# Lots of Predictions
#Returns Ratings(user, item, prediction)
pred = model.predictAll(pred_input) 

In [109]:
true_reorg = train.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred.map(lambda x:((x[0],x[1]), x[2]))

#Do the actual join
true_pred = true_reorg.join(pred_reorg)

#Need to be able to square root the Mean-Squared Error
from math import sqrt

MSE = true_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
RMSE = sqrt(MSE)#Results in 0.7629908117414474

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job 182 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:820)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:818)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:818)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1732)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:83)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1651)
	at org.apache.spark.SparkContext$$anonfun$stop$8.apply$mcV$sp(SparkContext.scala:1921)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1317)
	at org.apache.spark.SparkContext.stop(Spar

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 54296)
----------------------------------------


Traceback (most recent call last):
  File "/Users/drakaris/anaconda/envs/dlenv/lib/python3.5/socketserver.py", line 313, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/drakaris/anaconda/envs/dlenv/lib/python3.5/socketserver.py", line 341, in process_request
    self.finish_request(request, client_address)
  File "/Users/drakaris/anaconda/envs/dlenv/lib/python3.5/socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/drakaris/anaconda/envs/dlenv/lib/python3.5/socketserver.py", line 681, in __init__
    self.handle()
  File "/Users/drakaris/anaconda/envs/dlenv/lib/python3.5/site-packages/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/Users/drakaris/anaconda/envs/dlenv/lib/python3.5/site-packages/pyspark/serializers.py", line 577, in read_int
    raise EOFError
EOFError


In [81]:
print(RMSE)

0.4157597753887278


In [82]:
#Test Set Evaluation
#More dense, but nothing we haven't done before
test_input = test.map(lambda x:(x[0],x[1])) 
pred_test = model.predictAll(test_input)
test_reorg = test.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred_test.map(lambda x:((x[0],x[1]), x[2]))
test_pred = test_reorg.join(pred_reorg)
test_MSE = test_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
test_RMSE = sqrt(test_MSE)#1.0145549956596238

In [83]:
print(test_RMSE)

1.6774523890003226


In [ ]:
import random
random_set = random.sample(range(1, 9100000), 100000)
top_k_list=[[]]
for row in random_set: 
    top_k_list.append(model.recommendProducts(row,3))